In [62]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class ConvBNReLU(models.Sequential):
    def __init__(self, c_out, **kwargs):
        super().__init__([
            layers.Conv2D(c_out, kernel_size=3, strides=1, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.ReLU()
        ])

class ResidualBlock(tf.keras.Model):
    def __init__(self, in_channels, out_channels, strides=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.Activation('relu')
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()

        self.shortcut = tf.keras.Sequential()
        if strides != 1 or in_channels != out_channels:
            self.shortcut = tf.keras.Sequential([
                layers.Conv2D(out_channels, kernel_size=1, strides=strides, padding='same', use_bias=False),
                layers.BatchNormalization()
            ])

    def call(self, inputs):
        out = self.relu1(self.bn1(self.conv1(inputs)))
        out = self.bn2(self.conv2(out))
        shortcut = self.shortcut(inputs)
        
        out += shortcut
        out = tf.nn.relu(out)
        return out

def make_resnet(input_shape=(32, 32, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)

    # Initial convolution and batch normalization
    x = ConvBNReLU(64, input_shape=input_shape)(inputs)
    
    # Residual Blocks
    x = ResidualBlock(in_channels=64, out_channels=64, strides=1)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = ResidualBlock(in_channels=64, out_channels=128, strides=2)(x)

    x = ResidualBlock(in_channels=128, out_channels=256, strides=2)(x)

    x = ResidualBlock(in_channels=256, out_channels=512, strides=2)(x)

    # Global average pooling and dense layer for classification
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model
    
# Create and compile the model
model = make_resnet()
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv_bn_re_lu_24 (ConvBNReL  (None, 32, 32, 64)       1984      
 U)                                                              
                                                                 
 residual_block_51 (Residual  (None, 32, 32, 64)       74240     
 Block)                                                          
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 residual_block_52 (Residual  (None, 8, 8, 128)        230912    
 Block)                                                   

In [63]:
# Load and prepare the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [64]:
from PIL import Image, ImageEnhance, ImageOps
import random

class ShearX(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
            Image.BICUBIC, fillcolor=self.fillcolor)


class ShearY(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
            Image.BICUBIC, fillcolor=self.fillcolor)


class TranslateX(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, magnitude * x.size[0] * random.choice([-1, 1]), 0, 1, 0),
            fillcolor=self.fillcolor)


class TranslateY(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * x.size[1] * random.choice([-1, 1])),
            fillcolor=self.fillcolor)


class Rotate(object):
    # from https://stackoverflow.com/questions/
    # 5252170/specify-image-filling-color-when-rotating-in-python-with-pil-and-setting-expand
    def __call__(self, x, magnitude):
        rot = x.convert("RGBA").rotate(magnitude * random.choice([-1, 1]))
        return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(x.mode)


class Color(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Color(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Posterize(object):
    def __call__(self, x, magnitude):
        return ImageOps.posterize(x, magnitude)


class Solarize(object):
    def __call__(self, x, magnitude):
        return ImageOps.solarize(x, magnitude)


class Contrast(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Contrast(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Sharpness(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Sharpness(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Brightness(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Brightness(x).enhance(1 + magnitude * random.choice([-1, 1]))


class AutoContrast(object):
    def __call__(self, x, magnitude):
        return ImageOps.autocontrast(x)


class Equalize(object):
    def __call__(self, x, magnitude):
        return ImageOps.equalize(x)


class Invert(object):
    def __call__(self, x, magnitude):
        return ImageOps.invert(x)

In [65]:
class CIFAR10Policy(object):
    """ Randomly choose one of the best 25 Sub-policies on CIFAR10.

        Example:
        >>> policy = CIFAR10Policy()
        >>> transformed = policy(image)

        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     CIFAR10Policy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.1, "invert", 7, 0.2, "contrast", 6, fillcolor),
            SubPolicy(0.7, "rotate", 2, 0.3, "translateX", 9, fillcolor),
            SubPolicy(0.8, "sharpness", 1, 0.9, "sharpness", 3, fillcolor),
            SubPolicy(0.5, "shearY", 8, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.5, "autocontrast", 8, 0.9, "equalize", 2, fillcolor),

            SubPolicy(0.2, "shearY", 7, 0.3, "posterize", 7, fillcolor),
            SubPolicy(0.4, "color", 3, 0.6, "brightness", 7, fillcolor),
            SubPolicy(0.3, "sharpness", 9, 0.7, "brightness", 9, fillcolor),
            SubPolicy(0.6, "equalize", 5, 0.5, "equalize", 1, fillcolor),
            SubPolicy(0.6, "contrast", 7, 0.6, "sharpness", 5, fillcolor),

            SubPolicy(0.7, "color", 7, 0.5, "translateX", 8, fillcolor),
            SubPolicy(0.3, "equalize", 7, 0.4, "autocontrast", 8, fillcolor),
            SubPolicy(0.4, "translateY", 3, 0.2, "sharpness", 6, fillcolor),
            SubPolicy(0.9, "brightness", 6, 0.2, "color", 8, fillcolor),
            SubPolicy(0.5, "solarize", 2, 0.0, "invert", 3, fillcolor),

            SubPolicy(0.2, "equalize", 0, 0.6, "autocontrast", 0, fillcolor),
            SubPolicy(0.2, "equalize", 8, 0.6, "equalize", 4, fillcolor),
            SubPolicy(0.9, "color", 9, 0.6, "equalize", 6, fillcolor),
            SubPolicy(0.8, "autocontrast", 4, 0.2, "solarize", 8, fillcolor),
            SubPolicy(0.1, "brightness", 3, 0.7, "color", 0, fillcolor),

            SubPolicy(0.4, "solarize", 5, 0.9, "autocontrast", 3, fillcolor),
            SubPolicy(0.9, "translateY", 9, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.9, "autocontrast", 2, 0.8, "solarize", 3, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.1, "invert", 3, fillcolor),
            SubPolicy(0.7, "translateY", 9, 0.9, "autocontrast", 1, fillcolor)
        ]

    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment CIFAR10 Policy"

class SubPolicy(object):
    def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128, 128, 128)):
        ranges = {
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 150 / 331, 10),
            "translateY": np.linspace(0, 150 / 331, 10),
            "rotate": np.linspace(0, 30, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(int),
            "solarize": np.linspace(256, 0, 10),
            "contrast": np.linspace(0.0, 0.9, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.9, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,
            "invert": [0] * 10
        }

        func = {
            "shearX": ShearX(fillcolor=fillcolor),
            "shearY": ShearY(fillcolor=fillcolor),
            "translateX": TranslateX(fillcolor=fillcolor),
            "translateY": TranslateY(fillcolor=fillcolor),
            "rotate": Rotate(),
            "color": Color(),
            "posterize": Posterize(),
            "solarize": Solarize(),
            "contrast": Contrast(),
            "sharpness": Sharpness(),
            "brightness": Brightness(),
            "autocontrast": AutoContrast(),
            "equalize": Equalize(),
            "invert": Invert()
        }

        self.p1 = p1
        self.operation1 = func[operation1]
        self.magnitude1 = ranges[operation1][magnitude_idx1]
        self.p2 = p2
        self.operation2 = func[operation2]
        self.magnitude2 = ranges[operation2][magnitude_idx2]

    def __call__(self, img):
        if random.random() < self.p1:
            img = self.operation1(img, self.magnitude1)
        if random.random() < self.p2:
            img = self.operation2(img, self.magnitude2)
        return img

In [66]:
def apply_cutout(image, num_holes=1, max_h_size=8, max_w_size=8):
    """Applies Cutout augmentation to a single image."""
    # Convert PIL Image to numpy array
    image_np = np.array(image)
    
    h, w = image_np.shape[:2]
    mask = np.ones((h, w), np.float32)

    for _ in range(num_holes):
        y = np.random.randint(h)
        x = np.random.randint(w)
        
        y1 = np.clip(y - max_h_size // 2, 0, h)
        y2 = np.clip(y + max_h_size // 2, 0, h)
        x1 = np.clip(x - max_w_size // 2, 0, w)
        x2 = np.clip(x + max_w_size // 2, 0, w)

        mask[y1: y2, x1: x2] = 0.

    # Apply mask
    image_np = image_np * mask[:, :, np.newaxis]

    # Convert back to PIL Image
    return Image.fromarray(image_np.astype('uint8'))

In [67]:
def pad_image(image, pad_size=4, fill=0, padding_mode='reflect'):
    """Pad the given PIL Image on all sides with the given pad_size."""
    return ImageOps.expand(image, border=pad_size, fill=fill)

def random_crop(image, crop_size=(32, 32)):
    """Crop a random part of the image to the given size."""
    width, height = image.size
    new_width, new_height = crop_size

    left = np.random.randint(0, width - new_width + 1)
    top = np.random.randint(0, height - new_height + 1)

    image = image.crop((left, top, left + new_width, top + new_height))
    return image

In [68]:
def random_horizontal_flip(image, p=0.5):
    """Randomly flip the image horizontally with a probability of p."""
    if random.random() < p:
        return image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

def random_rotation(image, max_angle=20):
    """Randomly rotate the image within a given angle range."""
    angle = random.uniform(-max_angle, max_angle)
    return image.rotate(angle)

In [69]:
from tensorflow.keras.utils import Sequence
import numpy as np
import random

class CustomImageDataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size=64, augmentations=None):
        self.x_set = x_set
        self.y_set = y_set
        self.batch_size = batch_size
        self.augmentations = augmentations if augmentations else []

    def __len__(self):
        return np.ceil(len(self.x_set) / self.batch_size).astype(int)

    def __getitem__(self, idx):
        batch_x = self.x_set[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y_set[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        # Convert numpy arrays to PIL Images, apply augmentations, and convert back to numpy arrays
        x_batch_aug = np.array([self.apply_augmentations(Image.fromarray((image * 255).astype('uint8'))) for image in batch_x])
        
        # Convert PIL Images back to numpy arrays and normalize to [0, 1]
        x_batch_aug = np.array([np.array(image) for image in x_batch_aug]).astype('float32') / 255.0
        
        return x_batch_aug, batch_y

    def apply_augmentations(self, image):
        augmented_image = image
        for augmentation in self.augmentations:
            augmented_image = augmentation(augmented_image)
        return augmented_image

# Assuming CIFAR10Policy and apply_cutout are defined elsewhere
custom_augmentations = [pad_image, random_crop, random_horizontal_flip, random_rotation, CIFAR10Policy(), apply_cutout]

In [70]:
lr = 0.05
momentum = 0.9
weight_decay = 0.0005

optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr, 
    momentum=momentum, 
    nesterov=True,
    decay=weight_decay
)

# Compile the model with the updated learning_rate parameter
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [71]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback to save the model using the 'SavedModel' format
checkpoint = ModelCheckpoint('best_model', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_format='tf')

# Initialize the generator with the custom augmentations
custom_data_generator = CustomImageDataGenerator(x_train, y_train, batch_size=128, augmentations=custom_augmentations)

# Train the model using the custom data generator
history = model.fit(custom_data_generator,
                    steps_per_epoch=len(x_train) // 128,
                    epochs=250, 
                    validation_data=(x_test, y_test),
                    callbacks=[checkpoint])  # Include the checkpoint callback here

Epoch 1/250
390/390 [==============================] - ETA: 0s - loss: 1.9001 - accuracy: 0.3184
Epoch 1: val_accuracy improved from -inf to 0.44500, saving model to best_model


INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets


390/390 [==============================] - 19s 48ms/step - loss: 1.9001 - accuracy: 0.3184 - val_loss: 1.5079 - val_accuracy: 0.4450
Epoch 2/250
390/390 [==============================] - ETA: 0s - loss: 1.5290 - accuracy: 0.4466
Epoch 2: val_accuracy improved from 0.44500 to 0.56430, saving model to best_model


INFO:tensorflow:Assets written to: best_model\assets


...

390/390 [==============================] - 18s 46ms/step - loss: 0.4041 - accuracy: 0.8578 - val_loss: 0.3179 - val_accuracy: 0.9007
Epoch 145/250
390/390 [==============================] - ETA: 0s - loss: 0.3964 - accuracy: 0.8594
Epoch 145: val_accuracy did not improve from 0.90070
390/390 [==============================] - 16s 41ms/step - loss: 0.3964 - accuracy: 0.8594 - val_loss: 0.3139 - val_accuracy: 0.9006
Epoch 146/250
389/390 [============================>.] - ETA: 0s - loss: 0.3993 - accuracy: 0.8590
Epoch 146: val_accuracy did not improve from 0.90070
390/390 [==============================] - 15s 40ms/step - loss: 0.3992 - accuracy: 0.8591 - val_loss: 0.3217 - val_accuracy: 0.8956
Epoch 147/250
389/390 [============================>.] - ETA: 0s - loss: 0.3985 - accuracy: 0.8588
Epoch 147: val_accuracy did not improve from 0.90070
390/390 [==============================] - 16s 40ms/step - loss: 0.3986 - accuracy: 0.8587 - val_loss: 0.3257 - val_accuracy: 0.8981
Epoch 148/25

INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets


390/390 [==============================] - 21s 53ms/step - loss: 0.3921 - accuracy: 0.8614 - val_loss: 0.3097 - val_accuracy: 0.9011
Epoch 164/250
390/390 [==============================] - ETA: 0s - loss: 0.3866 - accuracy: 0.8627
Epoch 164: val_accuracy did not improve from 0.90110
390/390 [==============================] - 16s 40ms/step - loss: 0.3866 - accuracy: 0.8627 - val_loss: 0.3192 - val_accuracy: 0.8979
Epoch 165/250
390/390 [==============================] - ETA: 0s - loss: 0.3854 - accuracy: 0.8645
Epoch 165: val_accuracy did not improve from 0.90110
390/390 [==============================] - 16s 42ms/step - loss: 0.3854 - accuracy: 0.8645 - val_loss: 0.3228 - val_accuracy: 0.8989
Epoch 166/250
389/390 [============================>.] - ETA: 0s - loss: 0.3877 - accuracy: 0.8622
Epoch 166: val_accuracy did not improve from 0.90110
390/390 [==============================] - 17s 43ms/step - loss: 0.3877 - accuracy: 0.8622 - val_loss: 0.3149 - val_accuracy: 0.8998
Epoch 167/25

KeyboardInterrupt: 